In [69]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder().getOrCreate()
val df = spark.read.option("header","true").option("inferSchema","true").csv("/Users/rahul.bhatia/Desktop/scala-training/Netflix_2011_2016.csv")

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@64db79c
df: org.apache.spark.sql.DataFrame = [Date: string, Open: double ... 5 more fields]


In [70]:
df.show()

+----------+-----------------+------------------+----------+-----------------+---------+------------------+
|      Date|             Open|              High|       Low|            Close|   Volume|         Adj Close|
+----------+-----------------+------------------+----------+-----------------+---------+------------------+
|2011-10-24|       119.100002|120.28000300000001|115.100004|       118.839996|120460200|         16.977142|
|2011-10-25|        74.899999|         79.390001| 74.249997|        77.370002|315541800|11.052857000000001|
|2011-10-26|            78.73|         81.420001| 75.399997|        79.400002|148733900|         11.342857|
|2011-10-27|        82.179998| 82.71999699999999| 79.249998|80.86000200000001| 71190000|11.551428999999999|
|2011-10-28|        80.280002|         84.660002| 79.599999|84.14000300000001| 57769600|             12.02|
|2011-10-31|83.63999799999999|         84.090002| 81.450002|        82.080003| 39653600|         11.725715|
|2011-11-01|        80.10999

In [71]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [72]:
// Create a new dataframe with a column called HV Ratio that
// is the ratio of the High Price versus volume of stock traded
// for a day.
val df2 = df.withColumn("HV Ratio",df("High")/df("Volume"))

df2: org.apache.spark.sql.DataFrame = [Date: string, Open: double ... 6 more fields]


In [76]:
// What day had the Peak High in Price?
df.orderBy($"High".desc).show(1)

+----------+-----------------+----------+----------+----------+--------+------------------+
|      Date|             Open|      High|       Low|     Close|  Volume|         Adj Close|
+----------+-----------------+----------+----------+----------+--------+------------------+
|2015-07-13|686.6900019999999|716.159996|686.550026|707.610001|33205200|101.08714300000001|
+----------+-----------------+----------+----------+----------+--------+------------------+
only showing top 1 row



In [77]:
// What is the mean of the Close column?
df.select(mean("Close")).show()


+----------------+
|      avg(Close)|
+----------------+
|230.522453845909|
+----------------+



In [78]:
// What is the max and min of the Volume column?
df.select(max("Volume")).show()
df.select(min("Volume")).show()

+-----------+
|max(Volume)|
+-----------+
|  315541800|
+-----------+

+-----------+
|min(Volume)|
+-----------+
|    3531300|
+-----------+



In [80]:
// For Scala/Spark $ Syntax
import spark.implicits._

import spark.implicits._


In [81]:
// How many days was the Close lower than $ 600?
df.filter($"Close"<600).count()

res71: Long = 1218


In [82]:
// What percentage of the time was the High greater than $500 ?
(df.filter($"High">500).count()*1.0/df.count())*100

res72: Double = 4.924543288324067


In [83]:
// What is the Pearson correlation between High and Volume?
df.select(corr("High","Volume")).show()

+--------------------+
|  corr(High, Volume)|
+--------------------+
|-0.20960233287942157|
+--------------------+



In [85]:
// What is the max High per year?
val yeardf = df.withColumn("Year",year(df("Date")))
val yearmaxs = yeardf.select($"Year",$"High").groupBy("Year").max()
yearmaxs.select($"Year",$"max(High)").show()

+----+------------------+
|Year|         max(High)|
+----+------------------+
|2015|        716.159996|
|2013|        389.159988|
|2014|        489.290024|
|2012|        133.429996|
|2016|129.28999299999998|
|2011|120.28000300000001|
+----+------------------+



yeardf: org.apache.spark.sql.DataFrame = [Date: string, Open: double ... 6 more fields]
yearmaxs: org.apache.spark.sql.DataFrame = [Year: int, max(Year): int ... 1 more field]


In [86]:
// What is the average Close for each Calender Month?
val monthdf = df.withColumn("Month",month(df("Date")))
val monthavgs = monthdf.select($"Month",$"Close").groupBy("Month").mean()
monthavgs.select($"Month",$"avg(Close)").show()

+-----+------------------+
|Month|        avg(Close)|
+-----+------------------+
|   12| 199.3700942358491|
|    1|212.22613874257422|
|    6| 295.1597153490566|
|    3| 249.5825228971963|
|    5|264.37037614150944|
|    9|206.09598121568627|
|    4|246.97514271428562|
|    8|195.25599892727263|
|    7|243.64747528037387|
|   10|205.93297300900903|
|   11| 194.3172275445545|
|    2| 254.1954634020619|
+-----+------------------+



monthdf: org.apache.spark.sql.DataFrame = [Date: string, Open: double ... 6 more fields]
monthavgs: org.apache.spark.sql.DataFrame = [Month: int, avg(Month): double ... 1 more field]
